In [ ]:
import numpy as np
import cv2
import time
import os
from tensorflow import keras
from keras.models import load_model
model = load_model('traffic_classifier.h5')

cur_path = os.getcwd()
ImagesFilePath=os.path.join(cur_path,'meta',"16")
#ImageNamePath=os.listdir(ImagesFilePath)

def readImage(imagePath):
    img = cv2.imread(ImagesFilePath+'/'+imagePath,1)
    img = cv2.resize(img,(30,30))
    return img

def increaseContrast(img,alpha,beta):
	img=cv2.addWeighted(img,alpha,np.zeros(img.shape,img.dtype),0,beta)
	return img

def filteringImages(img):
    img=cv2.GaussianBlur(img,(11,11),0)
    return img


def returnRedness(img):
	yuv=cv2.cvtColor(img,cv2.COLOR_BGR2YUV)
	y,u,v=cv2.split(yuv)
	return v

def threshold(img,T=150):
	_,img=cv2.threshold(img,T,255,cv2.THRESH_BINARY)
	return img

def show(img):
	cv2.imshow('image',img)
	cv2.waitKey(0)
	cv2.destroyAllWindows()

def morphology(img,kernelSize=7):
	kernel = np.ones((kernelSize,kernelSize),np.uint8)
	opening = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
	return opening

def findContour(img):
	contours, hierarchy = cv2.findContours(img,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
	return contours

def findBiggestContour(contours):
	m=0
	c=[cv2.contourArea(i) for i in contours]
	return contours[c.index(max(c))]

def boundaryBox(img,contours):
	x,y,w,h=cv2.boundingRect(contours)
	img=cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
	sign=img[y:(y+h) , x:(x+w)]
	return img,sign

def preprocessingImageToClassifier(image=None,mu=89.77428691773054,std=70.85156431910688):
    image = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
    image = cv2.resize(image,(30,90))
    image = image.reshape(1,30,30,3)
    image = (image - mu) / std
    return image

def predict4(sign):
	img=preprocessingImageToClassifier(sign)
	return np.argmax(model.predict(img))

#def predict3(sign):
#	img=preprocessingImageToClassifier(sign,imageSize=32)
#	return np.argmax(model1.predict(img))

labelToText={0: 'Speed limit (20km/h)',
           1: 'Speed limit (30km/h)',
           2: 'Speed limit (50km/h)',
           3: 'Speed limit (60km/h)',
           4: 'Speed limit (70km/h)',
           5: 'Speed limit (80km/h)',
           6: 'End of speed limit (80km/h)',
           7: 'Speed limit (100km/h)',
           8: 'Speed limit (120km/h)',
           9: 'No passing',
           10: 'No passing veh over 3.5 tons',
           11: 'Right-of-way at intersection',
           12: 'Priority road',
           13: 'Yield',
           14: 'Stop',
           15: 'No vehicles',
           16: 'Veh > 3.5 tons prohibited',
           17: 'No entry',
           18: 'General caution',
           19: 'Dangerous curve left',
           20: 'Dangerous curve right',
           21: 'Double curve',
           22: 'Bumpy road',
           23: 'Slippery road',
           24: 'Road narrows on the right',
           25: 'Road work',
           26: 'Traffic signals',
           27: 'Pedestrians',
           28: 'Children crossing',
           29: 'Bicycles crossing',
           30: 'Beware of ice/snow',
           31: 'Wild animals crossing',
           32: 'End speed + passing limits',
           33: 'Turn right ahead',
           34: 'Turn left ahead',
           35: 'Ahead only',
           36: 'Go straight or right',
           37: 'Go straight or left',
           38: 'Keep right',
           39: 'Keep left',
           40: 'Roundabout mandatory',
           41: 'End of no passing',
           42: 'End no passing veh > 3.5 tons'}

import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.models import load_model

model = load_model('traffic_classifier.h5')

cap=cv2.VideoCapture('MVI_1049.avi')

while(True):
    _, frame = cap.read()
    img = returnRedness(frame) # step 1 --> specify the redness of the image
    img = threshold(img,T=150)
    try:
        contours = findContour(img)
        big = findBiggestContour(contours)
        if cv2.contourArea(big) > 100:
            img,sign = boundaryBox(frame,big)
            #cv2.putText(frame,labelToText[predict4(sign)], (20, 35),cv2.FONT_HERSHEY_PLAIN, 1, (255, 255, 255), 2)
            cv2.imshow('frame',frame)
    except:
        #cv2.putText(frame, labelToText[predict4(sign)], (20, 35), cv2.FONT_HERSHEY_PLAIN, 1, (255, 255, 255), 2)
        cv2.imshow('frame',frame)

    cv2.waitKey(0)
	#if cv2.waitKey(1) & 0xFF == ord('q'):
	#	break
#cap.release()
#cv2.destroyAllWindows()